# Reproducibility Experiment - Carl et al. (2020)

## Imports

In [1]:
from pathlib import Path
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import decode_predictions

2025-09-01 17:52:54.200511: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-01 17:52:54.203964: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-01 17:52:54.212173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756741974.225582  198924 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756741974.229299  198924 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756741974.240815  198924 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

## Setup
### Global variables

In [2]:
DATA_PATH = Path("data/kaggle-90-different-animals")

### Load the model

In [3]:
model = InceptionResNetV2(weights="imagenet")
target_size = model.input_shape[1:3] # the required image dimensions (299, 299)

2025-09-01 17:52:57.352617: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


### Load the images into a test data set
For the experiment, 90 common animals are chosen. They are sourced from Google Images and provided in a labeled format in a public [Kaggle dataset](https://www.kaggle.com/datasets/iamsouravbanerjee/animal-image-dataset-90-different-animals).

In [4]:
def load_normalized_image(path, target_size=target_size):
    image = Image.open(path).convert("RGB")
    image = image.resize(target_size)
    return np.array(image) / 255.0 # 1d array with floats from 0 to 1 as input for neural network

In [5]:
wildlife_images = []
labels = []

animal_species = sorted([d.name for d in DATA_PATH.iterdir() if d.is_dir()])

for species_name in animal_species:
    animal_image_folder = DATA_PATH / species_name # every species has its image folder
    for image_path in animal_image_folder.glob("*.jpg"):
        image_array = load_normalized_image(image_path)
        wildlife_images.append(image_array)
        labels.append(species_name)

X_test = np.stack(wildlife_images, axis=0) # there is no training phase, so all samples are test samples
y_true = np.array(labels)

## Test

In [7]:
y_pred = model.predict(X_test)

29/29 ━━━━━━━━━━━━━━━━━━━━ 127s 4s/step


In [8]:
y_pred_labels = [pred[0][1] for pred in decode_predictions(y_pred, top=1)]

In [9]:
y_pred_labels

['gazelle',
 'impala',
 'impala',
 'gazelle',
 'gazelle',
 'impala',
 'ibex',
 'gazelle',
 'impala',
 'impala',
 'badger',
 'badger',
 'badger',
 'badger',
 'American_black_bear',
 'badger',
 'badger',
 'badger',
 'badger',
 'badger',
 'hummingbird',
 'wood_rabbit',
 'hook',
 'hummingbird',
 'cowboy_boot',
 'barracouta',
 'house_finch',
 'chime',
 'tabby',
 'mink',
 'brown_bear',
 'brown_bear',
 'American_black_bear',
 'brown_bear',
 'American_black_bear',
 'brown_bear',
 'brown_bear',
 'American_black_bear',
 'brown_bear',
 'brown_bear',
 'bee',
 'bee',
 'honeycomb',
 'bee',
 'bee',
 'bee',
 'bee',
 'bee',
 'bee',
 'bee',
 'honeycomb',
 'ground_beetle',
 'fly',
 'ground_beetle',
 'cockroach',
 'leaf_beetle',
 'ground_beetle',
 'rhinoceros_beetle',
 'rhinoceros_beetle',
 'dung_beetle',
 'bison',
 'bison',
 'bison',
 'bison',
 'bison',
 'bison',
 'bison',
 'bison',
 'bison',
 'bison',
 'wild_boar',
 'wild_boar',
 'wild_boar',
 'wild_boar',
 'wild_boar',
 'wild_boar',
 'wild_boar',
 'wil

In [12]:
y_true

array(['antelope', 'antelope', 'antelope', 'antelope', 'antelope',
       'antelope', 'antelope', 'antelope', 'antelope', 'antelope',
       'badger', 'badger', 'badger', 'badger', 'badger', 'badger',
       'badger', 'badger', 'badger', 'badger', 'bat', 'bat', 'bat', 'bat',
       'bat', 'bat', 'bat', 'bat', 'bat', 'bat', 'bear', 'bear', 'bear',
       'bear', 'bear', 'bear', 'bear', 'bear', 'bear', 'bear', 'bee',
       'bee', 'bee', 'bee', 'bee', 'bee', 'bee', 'bee', 'bee', 'bee',
       'beetle', 'beetle', 'beetle', 'beetle', 'beetle', 'beetle',
       'beetle', 'beetle', 'beetle', 'beetle', 'bison', 'bison', 'bison',
       'bison', 'bison', 'bison', 'bison', 'bison', 'bison', 'bison',
       'boar', 'boar', 'boar', 'boar', 'boar', 'boar', 'boar', 'boar',
       'boar', 'boar', 'butterfly', 'butterfly', 'butterfly', 'butterfly',
       'butterfly', 'butterfly', 'butterfly', 'butterfly', 'butterfly',
       'butterfly', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat',
       'cat',